In [1]:
!pip install dash
!pip install ipython-sql
!pip install psycopg2-binary
!pip install pandas
!pip install numpy
!pip install numpy matplotlib
!pip install folium
!pip install geojson
!pip install jobspy
!pip install dash-bootstrap-components

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import pylab as pl
import folium
import json
import os
from folium import plugins
%matplotlib inline

import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_core_components as dcc

<ipython-input-2-284909b32e1d>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-284909b32e1d>:16: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())
car_brand = Data_to_Otomoto["car_brand"]
options = Data_to_Otomoto["options"]


In [4]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [5]:
cur = psql.cursor()
sql_line_graph = "SELECT AVG(cena), rok_produkcji, marka_pojazdu FROM otomoto_10 GROUP BY rok_produkcji, marka_pojazdu ORDER BY rok_produkcji DESC ;"
line_graph = sqlio.read_sql_query(sql_line_graph, psql)
sql_scatter_plot = "SELECT cena, przebieg, marka_pojazdu FROM otomoto_10;"
scatter_plot = sqlio.read_sql_query(sql_scatter_plot, psql)
conn = None


In [6]:
a = 0
for index, row in scatter_plot.iterrows():
    if (np.isnan(scatter_plot.at[index,'przebieg']) or np.isnan(scatter_plot.at[index,'cena'])):
        scatter_plot = scatter_plot.drop([index])
        a += 1
print("Drop rows where Nan from table otomoto: ", a)

Drop rows where Nan from table otomoto:  23


In [7]:
scatter_plot = scatter_plot.reset_index(drop=True)

In [ ]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.LITERA])

app.layout = html.Div(children=[
    html.H1('Cars for sell in Otomoto',style={'textAlign': 'center','front-size' :50}),
    dcc.Dropdown(id='brand_dropdown',
    options=options,

    optionHeight=35,                    #height/space between dropdown options
    value='BMW',                    #dropdown value selected automatically when page loads
    disabled=False,                     #disable dropdown value selection
    multi=False,                        #allow multiple dropdown values to be selected
    searchable=True,                    #allow user-searching of dropdown values
    search_value='',                    #remembers the value searched in dropdown
    placeholder='Please select...',     #gray, default text shown when no option is selected
    clearable=True,                     #allow user to removes the selected value
    style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
    # className='select_box',           #activate separate CSS document in assets folder
    # persistence=True,                 #remembers dropdown value. Used with persistence_type
    # persistence_type='memory'         #remembers dropdown value selected until...
    ),                                  #'memory': browser tab is refreshed
                                        #'session': browser tab is closed
                                        #'local': browser cookies are deleted
    
    html.Div(style={'textAlign': 'center','front-size' :25}, children=[
            html.Span('Number of years from which we have data: '),
            html.Span(id='total_rows_line_graph_data'),]),
    dcc.Graph(id='line_graph_fig'),
    html.Div(style={'textAlign': 'center','front-size' :25}, children=[
            html.Span('Number of cars used to generate graph: '),
            html.Span(id='total_rows_scatter_plot_data'),]),
    dcc.Graph(id='scatter_plot_fig')

    
    
    
    ])

@app.callback(
    [Output(component_id='line_graph_fig', component_property='figure'),
    Output(component_id='scatter_plot_fig', component_property='figure'),
    Output(component_id='total_rows_line_graph_data', component_property='children'),
    Output(component_id='total_rows_scatter_plot_data', component_property='children')],
    [Input(component_id='brand_dropdown', component_property='value')]
)

def build_graph(brand_chosen):
    
    line_graph_data = line_graph.loc[line_graph['marka_pojazdu'] == brand_chosen]
    line_graph_fig = go.Figure([go.Scatter(x=line_graph_data['rok_produkcji'], y=line_graph_data['avg'], 
                                mode='lines+markers')])
    
    scatter_plot_data = scatter_plot.loc[scatter_plot['marka_pojazdu'] == brand_chosen]
    scatter_plot_fig = px.scatter(scatter_plot_data, x=scatter_plot_data['przebieg'], 
                     y=scatter_plot_data['cena'], trendline="ols")
    
    total_rows_line_graph_data = len(line_graph_data.index) 
    print(total_rows_line_graph_data)
    total_rows_scatter_plot_data = len(scatter_plot_data.index) 
    print(total_rows_scatter_plot_data)
    
    return line_graph_fig, scatter_plot_fig, total_rows_line_graph_data, total_rows_scatter_plot_data


if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:8050/ (Press CTRL+C to quit)
192.168.10.24 - - [22/Oct/2021 22:15:52] "GET / HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /assets/stylesheet.css?m=1634931813.6475863 HTTP/1.1" 304 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_dash-layout HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:15:53] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.9/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/conda/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/conda/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "<ipython-input-8-80d206a8e

192.168.10.24 - - [22/Oct/2021 22:15:53] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.10.24 - - [22/Oct/2021 22:15:55] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.24 - - [22/Oct/2021 22:16:55] "POST /_dash-update-component HTTP/1.1" 200 -


24
301
